# import necassary libraries

In [86]:
from pathlib import Path
import pandas as pd
from email import policy
from email.parser import BytesParser
import os
import re
from bs4 import BeautifulSoup
import numpy as np
import nltk.corpus
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import DutchStemmer
import spacy
lemmaModel = spacy.load('nl_core_news_lg', disable = ['parser','ner'])

# set column width to maximum for better visibility of data
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jensk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jensk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Extract content from the emails

In [90]:
# define and print path to .eml files (emails)
pathString = os.getcwd() + '//BrainjarMails'
path = Path(pathString)
pathLength = len(pathString)
print(path)

# grab every file with the extension .eml
email_files = list(path.glob('*.eml'))

# create lists for the names and content of the emails + filecounter
names = []
contents = []
counter = 1
fileCount = len(email_files)

# loop over all found files
for email in email_files:
    
    #open each file in read bytes mode
    with open(email,'rb') as filepointer:
        
        # name is original filename minus the path and extension
        name = filepointer.name[pathLength:-4]
        
        # Parse data from email to message object
        message = BytesParser(policy=policy.default).parse(filepointer)
        
    # pass the plain text from the body of the email to a string variable. If no plain text is availible, 
    # just pass everything in the body
    try:
        content = message.get_body(preferencelist=('plain')).get_content()
    except:
        content = message.get_body().get_content()
    
    # Extract text from any HTML that is present.
    content = BeautifulSoup(content).get_text()
    
    # Remove escape characters (for example newlines)
    escapes = ''.join([chr(char) for char in range(1, 32)])
    #translator = str.maketrans(escapes, ' ')
    #content = content.translate(translator)
    content = re.sub(r'[' + escapes + r']',' ', content)
    
    # Remove any non-ascii characters
    content = content.encode('ascii', errors='ignore').decode()
    
    # Remove websites from mails (maybe not necassary)
    content = re.sub(r'http\S+', '', content)
    
    # Remove extra whitespaces
    content = re.sub(' +', ' ', content)
    
    # Remove excess non-alphanumeric characters except for punctuation
    # with punctuation
    #content = re.sub(r'[^A-Za-z0-9 ,?.:;!]+', '',content)
    # without punctuation
    content = re.sub(r'[^A-Za-z0-9 ]+', '',content)
    
    # additional filtering for privacy may be necessary
    content = re.sub(r'(BIC:) [A-Z]*','',content)
    content = re.sub(r'\w*\d\w*', '', content).strip()
    
    # remaining text to lowercase
    content = content.lower()
    
    # remove stopwords
    stop = stopwords.words('dutch')
    content =  " ".join([word for word in content.split() if word not in (stop)])
    
    # Stemming function
    def Stemmer(contentInput):
        tokenizedWords = word_tokenize(contentInput, language='dutch')
        stemmedContent = []
        stemmer = DutchStemmer()
        for word in tokenizedWords:
            stemmedContent.append(stemmer.stem(word))
            stemmedContent.append(" ")
        return "".join(stemmedContent)
       
    # Lemmatization function
    def Lemmatizer(contentInput):
        document = lemmaModel(contentInput)
        return " ".join([token.lemma_ for token in document])
    
    # Stemming or lemmatization
    #content = Stemmer(content)
    content = Lemmatizer(content)
    
    # add name and content of current email to their respective lists
    names.append(name)
    contents.append(content)
    
    #close the current file
    filepointer.close()
    
    # filecounter
    print("Counter: " + str(counter) + '/' + str(fileCount), end="\r")
    counter += 1

C:\De Nayer 2022-2023\Bachelorproef\Jupyter notebook\BrainjarMails


C:\ProgramData\Miniconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


### Turn lists into dataframe for easy exploration

In [91]:
dfNames = pd.DataFrame([names, contents]).T
dfNames.columns = ['names', 'contents']

### Set class index based on title

In [92]:
dfNames['classIndex'] = 0
dfNames['classIndex'] = np.where(dfNames['names'].str.contains('facturen'), 1, dfNames['classIndex'])
dfNames['classIndex'] = np.where(dfNames['names'].str.contains('aanmaningen'), 2, dfNames['classIndex'])

### Display top 20 rows

In [93]:
dfNames.head(20)

,names,contents,classIndex
0,00057d8d-2e28-45d2-8836-e80003cadafa-andere,bposebilemail klik online versie bijlage bpost nv publiek recht factuur nummer online beschikbaar goed klant wij informeren bijlage bpost nv publiek recht factuur nummer dd vanaf online beschikbaar bijlage downloaden vriendelijk groet facturatiedienst geval vragenbetwisting betrekking bijlage gelief contacteren via,0
1,00403521-b493-413d-b0a9-db90dd069dad-facturen,no Title Given gelief factuur bijlage plaatsvinden vraag contacteer businessbrailbe,1
2,005385fa-959e-4a8b-8763-15261c217b41-facturen,goed bijlage factuur maand juli vinden vriendelijk groet nadine haij Fleet Finance coordinator Nhaijencallexcellcom Tiensesteenweg bus blok c sinttruiden Wwwcallexcellcom,1
3,00552116-d17a-452a-b1f3-e69f76a7cef1-facturen,geacht klant bijlage ontvangen ons factuur vraag kunnen terecht n ons medewerker via telefoonnummer email Debiteurenfacilicombe vriendelijk groet gom nv administratief postbus fin facturatie facilicom services Group Belgium Noorderplaats Antwerpent m finvoicefacilicombe Wwwfacilicombeplease consider the environment before printing this email,1
4,00680439-07fe-4e6d-a145-e339a3d73e40-facturen,bericht bevatten elektronisch factuur elektronisch ondertekenen pdf document wettelijk factuur kader wetgeving elektronisch factureren zijn verplichten factur origineel elektronisch formaat bewaren gedurende wettelijk bepalen periode geprinte versie elektronisch factuur gelden geval wettelijk factuur elektronisch ondertekenen factuur bekijken valideren dienen Adobe reader hoog gebruiken factuurnummer factuurdatum dienstverlener basware Belgium nv verzender bright plusvriendelijk Groetenbright plus please do not respond to this email this Message sent automatically and responses to it can not be processed for further information on our services and contact detail please visit baswarebe,1
5,0070566b-63e7-4cca-82ff-116f1f834130-andere,bedanken reservatie definitief reservatie voltooi volgen stap ophaling materiaal stap contract controleer contract nauwkeurig onderteken digitaal via link stap id klik hieronder naam identiteit verifiren meerdere contact verifieren hoofdverantwoordelijk sam Haeghens staan naam groen hoeven stap betaling betalen factuur termijn dag nuttig info komen ophalen stuur mail ophaler bewijs materiaal oppikken Gent volgens uur aanduiden contract al stap moeten voltooien factuur ontvangen per mail inlevering controle materiaal bedanken ostron team definitief reservatie annuleren enkel kosteloos binnen verzending Mail daarna kost verbinden zien ons algemeen voorwaarde detail to change your email preferences when comments are added to this discussion click here sent from,0
6,0076fa2e-db45-4b1f-bd08-dc73ae26b42b-andere,hoi toevoegen jullie daan Ceulemans toe leverancier aub firmanaam kadanz kil Niel Daankadanzcom btw nr groetjes hillen sijber planner image dpg medium Medialaan vilvoorde t m Hildesijbersdpgmediabe dpgmediabe,0
7,007e144d-da55-43e1-938c-d94ee76d43c7-aanmaningen,goed bijlage ons ruilfactuur factuur jullie hierop betrekking Mvg Bjrge ponnet boekhouder t f uitgeverij Lannoo nv Kasteelstraat tielt belgi t bekijk ons catalogus lannoocom facebook instagram twitter linkedin this email may contain confidential material if you were not an intended recipient please notify the sender and delete all copie we may monitor email to and from our network lannoo publishers ltd Jerry Excelmans verzonden vrijdag augustus Bjrge ponnet onderwerp re fw reminder openstaan factur uitgeverij Lannoo nv Dpg media nv be goed Bjrge we factuur waarvan collega kristof spreken mogen ontvangen mogen Suppliersdpgmediabe verzonden kopie mail graag alvast bedanken vriendelijk groet jerry gl accountant dpg media Mediaplein Antwerpen Jerryexcelmansdpgmediabe dpgmediabe on mon jun at Bjrge ponnet wrote Hallo kristof terugbetaling vrijdag uitvoeren hoogstwaarschijnlijk vandaag jullie toekomen tegenfactuur excl btw achter collega ver zitten fijn dag Mvg Bjrge ponnet boekhou

### Save dataframe to csv file

In [94]:
dfNames.to_csv('test_extraction_emails.csv')

#### Sources:
- https://stackoverflow.com/questions/8115261/how-to-remove-all-the-escape-sequences-from-a-list-of-strings
- https://enjoylifescience.com/2020/11/05/analyzing-emails-in-python/
- https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
- https://towardsdatascience.com/remove-personal-information-from-text-with-python-232cb69cf074
- https://monkeylearn.com/blog/text-cleaning/#:~:text=Text%20cleaning%20can%20be%20performed,words%20to%20their%20root%20form.&text=You'd%20need%20to%20perform,Removing%20Stopwords
- https://www.datacamp.com/tutorial/stemming-lemmatization-python
- https://www.projectpro.io/recipes/use-spacy-lemmatizer